In [1]:
import pandas as pd

df = pd.read_csv('sub_tweets_preprocessed_NO_NER.csv')
df.head(-5)

,tweet_id,text,account,domain,background,label
0,1589375766638522368,mirthg46 sainzdlmaza ya no hay previsto nada e...,@perezreverte,PERSON,ART,ES
1,1492150578738257921,con mañueco pedimos bajar impuestos ayudas par...,@PabloCasado_,PERSON,POL,ES
2,1608890157008035845,la fiscalía de valencia considera que los hech...,@el_pais,ORG,NEW,ES
3,1602619348744445953,qatar2022 🗓 semifinales ⚽ argentina 🇦🇷 croacia...,@Argentina,PERSON,SPORT,ARG
4,1599335796128956416,los jugadores aprendemos de los errores,@SEFutbol,ORG,NEW,ARG
...,...,...,...,...,...,...
9966,1547517129162907649,sánchez los había entregado para estar un rato...,@Santi_ABASCAL,PERSON,POL,ES
9967,1608174829630328832,ocurre algo similar en extremadura en castilla...,@eldiarioes,ORG,NEW,ES
9968,1608255758935752705,la estrenaron hace cinco años pasó desapercibi...,@LANACION,ORG,NEW,ARG
9969,1608555809662992386,🗣️ neymar antes de pelé el fútbol era apenas u...,@Ambitocom,ORG,NEW,ARG


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    DistilBertConfig,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
from datasets.load import load_dataset, load_metric
import os
from transformers import AdamW, get_scheduler



# Opening Dataset
df = pd.read_csv('sub_tweets_preprocessed_NO_NER.csv')

# Load the dataset from the Hugging Face datasets library
dataset = load_dataset('csv', data_files='sub_tweets_preprocessed_NO_NER.csv', delimiter=',')

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/distilbert-base-spanish-uncased")

# Encoding Data
def process_data(row):
    text = row['text']
    text = str(text)
    text = ' '.join(text.split())
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = row['label']
    if label == 'ES':
        label = 1
    else:
        label = 0
    encodings['label'] = label
    encodings['text'] = text
    return encodings

processed_data = [process_data(row) for _, row in df.iterrows()]

# Shuffle the dataset
import random
random.shuffle(processed_data)

# Splitting Dataset
train, test = train_test_split(processed_data, test_size=0.2)


train_dict = {k: [example[k] for example in train] for k in train[0]}
test_dict = {k: [example[k] for example in test] for k in test[0].keys()}


# Config and Model
config = DistilBertConfig.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", num_labels=2, output_hidden_states=True
)
model = DistilBertForSequenceClassification.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", config=config,
)

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Metric and Metric computation function
metric = load_metric("f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    f1 = metric.compute(predictions=preds, references=labels, average="weighted")
    return f1


# Training Arguments
training_args = TrainingArguments(
    output_dir="./new_stuff_NO_NER",
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluation
eval_results = trainer.predict(test_dataset)
eval_metric = compute_metrics(eval_results)
print(eval_metric)


Found cached dataset csv (C:/Users/LENOVO/.cache/huggingface/datasets/csv/default-0b7a1ae448fd526a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['classifier.weight'

Step,Training Loss,Validation Loss,F1
10,0.687000,0.680435,0.434084
20,0.671700,0.666454,0.543940
30,0.631900,0.650574,0.592764
40,0.665000,0.627392,0.660843
50,0.643000,0.624815,0.658815
60,0.599000,0.591489,0.697949
70,0.612300,0.572906,0.704675
80,0.596300,0.573107,0.690678
90,0.565100,0.546986,0.718202
100,0.546900,0.538613,0.724124


{'f1': 0.819616099389951}
